Ejercicio: construir una matriz de confusión

En este ejercicio, profundizamos en la medición del rendimiento de los modelos de clasificación, utilizando los conceptos de conjuntos de datos desequilibrados, precisión y matrices de confusión.

Visualización de datos
Nuestro nuevo conjunto de datos representa diferentes clases de objetos que se encuentran en la nieve.

Comencemos este ejercicio cargando y echando un vistazo a nuestros datos:

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

#Let's have a look at the data
dataset

Exploración de datos

Podemos ver que el conjunto de datos tiene datos continuos (tamaño, rugosidad, movimiento) y categóricos (color y etiqueta). Hagamos una exploración rápida de datos y veamos qué diferentes clases de etiquetas tenemos y sus respectivos conteos:

In [ ]:
import graphing # custom graphing code. See our GitHub repo for details

# Plot a histogram with counts for each label
graphing.multiple_histogram(dataset, label_x="label", label_group="label", title="Label distribution")

El histograma de arriba hace que sea muy fácil entender tanto las etiquetas que tenemos en el conjunto de datos como su distribución.

Una información importante a tener en cuenta es que este es un conjunto de datos desequilibrado: las clases no están representadas en la misma proporción (tenemos 4 veces más rocas y árboles que animales, por ejemplo).

Esto es relevante ya que los conjuntos desequilibrados son muy comunes "en la naturaleza", y en el futuro aprenderemos cómo abordar eso para construir mejores modelos.

Podemos hacer el mismo análisis para la función de color:

In [ ]:
# Plot a histogram with counts for each label
graphing.multiple_histogram(dataset, label_x="color", label_group="color", title="Color distribution")

Podemos notar que:

Tenemos 8 categorías de colores diferentes.<br>
La función de color también está muy desequilibrada.<br>
El algoritmo de trazado no es lo suficientemente inteligente como para asignar los colores correctos a sus respectivos nombres.<br>
Veamos qué podemos encontrar sobre las otras características:

In [ ]:
graphing.box_and_whisker(dataset, label_y="size", title='Boxplot of "size" feature')

Arriba podemos ver que la mayoría de nuestras muestras son relativamente pequeñas, con tamaños que van de 0 a 70, pero tenemos algunos valores atípicos mucho más grandes.

Echemos un vistazo a la función de rugosidad:

In [ ]:
graphing.box_and_whisker(dataset, label_y="roughness", title='Boxplot of "roughness" feature')

No hay mucha variación aquí: los valores de rugosidad oscilan entre 0 y un poco más de 2, y la mayoría de las muestras tienen valores cercanos a la media.

Finalmente, tracemos la función de movimiento:

In [ ]:
graphing.box_and_whisker(dataset, label_y="motion", title='Boxplot of "motion" feature')

La mayoría de los objetos parecen estar estáticos o moverse muy lentamente. Hay una cantidad menor de objetos que se mueven más rápido, con un par de valores atípicos por encima de 10.

A partir de los datos anteriores, se podría suponer que los objetos más pequeños y rápidos son probablemente excursionistas y animales, mientras que los elementos más grandes y estáticos son árboles y rocas.

Construcción de un modelo de clasificación

Construyamos y entrenemos un modelo de clasificación utilizando un bosque aleatorio, para predecir la clase de un objeto en función de las características de nuestro conjunto de datos:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=2)
print(train.shape)
print(test.shape)

Ahora podemos entrenar nuestro modelo, usando el conjunto de datos de tren que acabamos de crear:

In [ ]:
# Create the model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used (leave color out for now)
features = ["size", "roughness", "motion"]

# Train the model
model.fit(train[features], train.label)

print("Model trained!")

Evaluando nuestro modelo
Ahora podemos usar nuestro modelo recién entrenado para hacer predicciones usando el conjunto de prueba.

Al comparar los valores predichos con las etiquetas reales (también llamados valores verdaderos), podemos medir el rendimiento del modelo usando diferentes métricas.

La precisión, por ejemplo, es simplemente el número de etiquetas predichas correctamente de todas las predicciones realizadas:

In [ ]:
# Import a function that measures a models accuracy
from sklearn.metrics import accuracy_score

# Calculate the model's accuracy on the TEST set
actual = test.label
predictions = model.predict(test[features])

# Return accuracy as a fraction
acc = accuracy_score(actual, predictions)

# Return accuracy as a number of correct predictions
acc_norm = accuracy_score(actual, predictions, normalize=False)

print(f"The random forest model's accuracy on the test set is {acc:.4f}.")
print(f"It correctly predicted {acc_norm} labels in {len(test.label)} predictions.")

¡Nuestro modelo parece estar haciéndolo bastante bien!

Esa intuición, sin embargo, puede ser engañosa:

La precisión no tiene en cuenta las predicciones erróneas realizadas por el modelo

Tampoco es muy bueno para pintar una imagen clara en conjuntos de datos de clases desequilibradas, como el nuestro, donde la cantidad de clases posibles no está distribuida uniformemente (recuerde que tenemos 800 árboles, 800 rocas, pero solo 200 animales)

Construyendo una matriz de confusión
Una matriz de confusión es una tabla donde comparamos las etiquetas reales con lo que predijo el modelo. Nos da una comprensión más detallada de cómo está funcionando el modelo y dónde está haciendo las cosas bien o fallando.

Esta es una de las formas en que podemos hacer eso en el código:

In [ ]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

print("Confusion matrix for the test set:")
print(cm)


Si bien la matriz anterior puede ser útil en los cálculos, no es muy útil ni intuitiva.

Agreguemos un gráfico con etiquetas y colores para convertir esos datos en información real:

In [ ]:
# We use plotly to create plots and charts
import plotly.figure_factory as ff

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['animal', 'hiker', 'rock', 'tree']
x = y = sorted(list(test["label"].unique()))

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=100, b=50))
fig['data'][0]['showscale'] = True
fig.show()

Observe que el gráfico tiene las etiquetas Real en el eje y y las etiquetas Predicho en el eje x, según lo definido por la llamada a la función confusion_matrix.

Podemos ver que el modelo es generalmente preciso, pero solo porque tenemos muchas rocas y árboles en nuestro conjunto y porque funciona bien en esas clases.

Cuando se trata de excursionistas y animales, el modelo se confunde (muestra una gran cantidad de FP y FN), pero debido a que estas clases están menos representadas en el conjunto de datos, la puntuación de precisión sigue siendo alta.

Resumen

En este ejercicio hablamos de los siguientes conceptos:

Conjuntos de datos desequilibrados, donde las características o clases pueden representarse mediante un número desproporcionado de muestras.<br>
La precisión como métrica para evaluar el rendimiento del modelo y sus deficiencias.<br>
Cómo generar, graficar e interpretar matrices de confusión, para comprender mejor cómo funciona un modelo de clasificación.